# Data Cleaning

## Imports

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
import re

## Sources

### I. National Center for Education Statistic (NCES) Elementary/Secondary Information System (ElSi) 

Source: https://nces.ed.gov/ccd/elsi/

Table generator: https://nces.ed.gov/ccd/elsi/tableGenerator.aspx 
- (Helpful for pre-aggregation of data because it pulls from multiple sources)

Metrics pulled for fiscal years 2011-2012 to 2021-2022 years:
- Table ID: 655145
- Agency Name
- State Name [District]
- State Abbr [District]
- Agency ID - NCES Assigned [District]
- County Name [District]
- ANSI/FIPS State Code [District]
- Agency Type [District]
- School District Level Code (SCHLEV) [District Finance]
- Locale [District]
- Latitude [District]
- Longitude [District]
- State Agency ID [District]
- Census ID (CENSUSID) [District Finance]
- Highest Grade Offered [District]
- Total Students All Grades (Excludes AE) [District]
- Total Students All Grades (Includes AE) [District]
- Limited English Proficient (LEP) / English Language Learners (ELL) [District]
- Male Students [District]
- Female Students [District]
- American Indian/Alaska Native Students [District]
- Asian or Asian/Pacific Islander Students [District]
- Hispanic Students [District]
- Black or African American Students [District]
- White Students [District]
- Nat. Hawaiian or Other Pacific Isl. Students [District]
- Two or More Races Students [District]
- Total Race/Ethnicity [District]
- Full-Time Equivalent (FTE) Teachers [District]
- Pupil/Teacher Ratio [District]
- Paraprofessionals/Instructional Aides [District]
- LEA Administrative Support Staff [District]
- School Administrators [District]
- Total Revenue (TOTALREV) [District Finance]
- Total Revenue - Local Sources (TLOCREV) [District Finance]
- Total Revenue - State Sources (TSTREV) [District Finance]
- Total Revenue - Federal Sources (TFEDREV) [District Finance]
- Total Current Expenditures - Instruction (TCURINST) [District Finance]
- Total Current Expenditures - Support Services (TCURSSVC) [District Finance]
- Total Current Expenditures - Salary (Z32) [District Finance]
- Total Expenditures (TOTALEXP) [District Finance]
- Total Expenditures - Capital Outlay (TCAPOUT) [District Finance]
- Total Current Expenditures - Instruction (TCURINST) per Pupil (V33) [District Finance]
- Total Current Expenditures - Support Services (TCURSSVC) per Pupil (V33) [District Finance]
- Total Current Expenditures - Other El-Sec Programs (TCUROTH) per Pupil (V33) [District Finance]
- Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance]
- Long Term Debt - Outstanding at End of FY (_41F) [District Finance]
- Debt Service Funds (W01) [District Finance]

In [2]:
df = pd.read_csv(
    "raw/ELSI_csv_export_6389659429608995655097.csv",
    skiprows=6,
    skipfooter=8,
    engine="python",
)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146 entries, 0 to 1145
Columns: 461 entries, Agency Name to Debt Service Funds (W01) [District Finance] 2011-12
dtypes: int64(2), object(459)
memory usage: 4.0+ MB


In [4]:
# Data has strange values used for NaN so replacing all of those
df.replace(to_replace=["†","–","‡"], value=np.nan, inplace=True)

/tmp/ipykernel_1480502/777335771.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace=["†","–","‡"], value=np.nan, inplace=True)


In [5]:
# Make a singular county coumn for each district based on latest available
county_cols = [
    "County Name [District] 2021-22",
    "County Name [District] 2020-21",
    "County Name [District] 2019-20",
    "County Name [District] 2018-19",
    "County Name [District] 2017-18",
    "County Name [District] 2016-17",
    "County Name [District] 2015-16",
    "County Name [District] 2014-15",
    "County Name [District] 2013-14",
    "County Name [District] 2012-13",
    "County Name [District] 2011-12"
]

df["County Name [District] Latest available year"] = df[county_cols].bfill(axis=1).iloc[:, 0]

df.drop(columns=county_cols, inplace=True)

In [6]:
# Drop this column I didn't mean to download
cols_to_drop = [
    "Total Students All Grades (Includes AE) [District] 2021-22",
    "Total Students All Grades (Includes AE) [District] 2020-21",
    "Total Students All Grades (Includes AE) [District] 2019-20",
    "Total Students All Grades (Includes AE) [District] 2018-19",
    "Total Students All Grades (Includes AE) [District] 2017-18",
    "Total Students All Grades (Includes AE) [District] 2016-17",
    "Total Students All Grades (Includes AE) [District] 2015-16",
    "Total Students All Grades (Includes AE) [District] 2014-15",
]

df.drop(columns=cols_to_drop, inplace=True)

In [7]:
for col in df.columns:
    print(col)

Agency Name
State Name [District] Latest available year
State Abbr [District] Latest available year
Agency ID - NCES Assigned [District] Latest available year
ANSI/FIPS State Code [District] Latest available year
Agency Type [District] 2021-22
Agency Type [District] 2020-21
Agency Type [District] 2019-20
Agency Type [District] 2018-19
Agency Type [District] 2017-18
Agency Type [District] 2016-17
Agency Type [District] 2015-16
Agency Type [District] 2014-15
Agency Type [District] 2013-14
Agency Type [District] 2012-13
Agency Type [District] 2011-12
School District Level Code (SCHLEV) [District Finance] 2021-22
School District Level Code (SCHLEV) [District Finance] 2020-21
School District Level Code (SCHLEV) [District Finance] 2019-20
School District Level Code (SCHLEV) [District Finance] 2018-19
School District Level Code (SCHLEV) [District Finance] 2017-18
School District Level Code (SCHLEV) [District Finance] 2016-17
School District Level Code (SCHLEV) [District Finance] 2015-16
Schoo

In [8]:
# Change data from wide to long format
wide_cols = [
"Agency Type [District]",
"School District Level Code (SCHLEV) [District Finance]",
"Locale [District]",
"Latitude [District]",
"Longitude [District]",
"State Agency ID [District]",
"Census ID (CENSUSID) [District Finance]",
"Highest Grade Offered [District]",
"Total Students All Grades (Excludes AE) [District]",
"Limited English Proficient (LEP) / English Language Learners (ELL) [District]",
"Male Students [District]",
"Female Students [District]",
"American Indian/Alaska Native Students [District]",
"Asian or Asian/Pacific Islander Students [District]",
"Hispanic Students [District]",
"Black or African American Students [District]",
"White Students [District]",
"Nat. Hawaiian or Other Pacific Isl. Students [District]",
"Two or More Races Students [District]",
"Total Race/Ethnicity [District]",
"Full-Time Equivalent (FTE) Teachers [District]",
"Pupil/Teacher Ratio [District]",
"Paraprofessionals/Instructional Aides [District]",
"LEA Administrative Support Staff [District]",
"School Administrators [District]",
"Total Revenue (TOTALREV) [District Finance]",
"Total Revenue - Local Sources (TLOCREV) [District Finance]",
"Total Revenue - State Sources (TSTREV) [District Finance]",
"Total Revenue - Federal Sources (TFEDREV) [District Finance]",
"Total Current Expenditures - Instruction (TCURINST) [District Finance]",
"Total Current Expenditures - Support Services (TCURSSVC) [District Finance]",
"Total Current Expenditures - Salary (Z32) [District Finance]",
"Total Expenditures (TOTALEXP) [District Finance]",
"Total Expenditures - Capital Outlay (TCAPOUT) [District Finance]",
"Total Current Expenditures - Instruction (TCURINST) per Pupil (V33) [District Finance]",
"Total Current Expenditures - Support Services (TCURSSVC) per Pupil (V33) [District Finance]",
"Total Current Expenditures - Other El-Sec Programs (TCUROTH) per Pupil (V33) [District Finance]",
"Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance]",
"Long Term Debt - Outstanding at End of FY (_41F) [District Finance]",
"Debt Service Funds (W01) [District Finance]"
]

df["id"] = df.index

long_df = pd.wide_to_long(df=df,
                          stubnames=wide_cols,
                          i="id",
                          j="Year",
                          sep=" ",
                          suffix=r"\d{4}-\d{2}").reset_index()

long_df.drop(columns=["id"], inplace=True)

In [9]:
long_df["Highest Grade Offered [District]"].head(20)

0              NaN
1        8th Grade
2       12th Grade
3       10th Grade
4        7th Grade
5        8th Grade
6              NaN
7        8th Grade
8        9th Grade
9       12th Grade
10      12th Grade
11      12th Grade
12       8th Grade
13    Kindergarten
14       8th Grade
15       8th Grade
16       8th Grade
17       8th Grade
18      12th Grade
19      12th Grade
Name: Highest Grade Offered [District], dtype: object

In [10]:
# Only look at districts with a high school, i.e. highest grade is 12
#df_filtered = long_df[long_df["Highest Grade Offered [District]"] == "12th Grade"]
#df_filtered["Highest Grade Offered [District]"].head(20)

# Do this before plotting

In [11]:
# Renmaing columns for easier understanding
df_filtered = long_df.rename(columns={
    "Agency Name": "School District",
    "State Name [District] Latest available year": "State Name",
    "State Abbr [District] Latest available year": "State",
    "County Name [District] Latest available year": "County Name",
    "Agency ID - NCES Assigned [District] Latest available year": "nces_id",
    "ANSI/FIPS State Code [District] Latest available year": "ansi_fips",
    "Census ID (CENSUSID) [District Finance]": "census_id"
})

# Get rid of the '[District]' portions of the column names
df_filtered.columns = df_filtered.columns.str.replace(r"\s*\[.*?\]", "", regex=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12606 entries, 0 to 12605
Data columns (total 47 columns):
 #   Column                                                                        Non-Null Count  Dtype 
---  ------                                                                        --------------  ----- 
 0   Year                                                                          12606 non-null  object
 1   ansi_fips                                                                     12606 non-null  int64 
 2   nces_id                                                                       12606 non-null  int64 
 3   School District                                                               12606 non-null  object
 4   County Name                                                                   12452 non-null  object
 5   State                                                                         12606 non-null  object
 6   State Name                            

In [12]:
# Make numeric columns numeric
numeric_cols = [
    'Total Students All Grades (Excludes AE)',
    'Limited English Proficient (LEP) / English Language Learners (ELL)',
    'Male Students',
    'Female Students',
    'Total Race/Ethnicity',
    'White Students',
    'Black or African American Students',
    'American Indian/Alaska Native Students',
    'Asian or Asian/Pacific Islander Students',
    'Hispanic Students',
    'Nat. Hawaiian or Other Pacific Isl. Students',
    'Two or More Races Students',
    'Full-Time Equivalent (FTE) Teachers',
    'Pupil/Teacher Ratio',
    'Paraprofessionals/Instructional Aides',
    'LEA Administrative Support Staff',
    'School Administrators',
    'Total Revenue (TOTALREV)',
    'Total Revenue - Local Sources (TLOCREV)',
    'Total Revenue - State Sources (TSTREV)',
    'Total Revenue - Federal Sources (TFEDREV)',
    'Total Current Expenditures - Instruction (TCURINST)',
    'Total Current Expenditures - Support Services (TCURSSVC)',
    'Total Current Expenditures - Salary (Z32)',
    'Total Expenditures (TOTALEXP)',
    'Total Expenditures - Capital Outlay (TCAPOUT)',
    'Total Current Expenditures - Instruction (TCURINST) per Pupil (V33)',
    'Total Current Expenditures - Support Services (TCURSSVC) per Pupil (V33)',
    'Total Current Expenditures - Other El-Sec Programs (TCUROTH) per Pupil (V33)',
    'Total Expenditures (TOTALEXP) per Pupil (V33)',
    'Long Term Debt - Outstanding at End of FY (_41F)',
    'Debt Service Funds (W01)'
]

for col in numeric_cols:
    df_filtered[col] = pd.to_numeric(df_filtered[col])

In [13]:
# Making the race/ethnicity & gender columns percentages of total

def make_percentage(your_data: pd.DataFrame,
                    col: str,
                    total: str):
    """Make a percentage column for a given raw value"""

    your_data["Percent " + col] = your_data[col]/your_data[total]

    your_data.drop(columns=[col], inplace=True)

In [14]:
percent_cols = {'Male Students': 'Total Students All Grades (Excludes AE)',
                'Female Students': 'Total Students All Grades (Excludes AE)',
                'Limited English Proficient (LEP) / English Language Learners (ELL)': 'Total Students All Grades (Excludes AE)',
                'White Students': 'Total Race/Ethnicity',
                'Black or African American Students': 'Total Race/Ethnicity',
                'American Indian/Alaska Native Students': 'Total Race/Ethnicity',
                'Asian or Asian/Pacific Islander Students': 'Total Race/Ethnicity',
                'Hispanic Students': 'Total Race/Ethnicity',
                'Nat. Hawaiian or Other Pacific Isl. Students': 'Total Race/Ethnicity',
                'Two or More Races Students': 'Total Race/Ethnicity'
                }

for col, total in percent_cols.items():
    make_percentage(df_filtered, col, total)

df_filtered.drop(columns=['Total Race/Ethnicity'], inplace=True)

In [15]:
district = df_filtered.reset_index(drop=True)

In [16]:
district.to_csv("clean/characteristics.csv", index=False)

### II. New York State Education Department (NYSED) 

Source: https://data.nysed.gov/downloads.php

In [17]:
def fix_percent(grad_filt: pd.DataFrame, col: str):
    grad_filt[col] = grad_filt[col].replace(["-", "#"], value=np.nan)

    if grad_filt[col].dtype in ["int64", "float64"]:
        grad_filt = grad_filt.rename(columns={col: col[0] + "rate"})
    else:
        grad_filt[col[0] + "rate"] = (
            pd.to_numeric(grad_filt[col].str.rstrip("%"), errors="coerce") / 100
        )

        grad_filt.drop(columns=[col], inplace=True)

In [18]:
def clean_newer_data(file: str):
    
    grad = pd.read_csv(file)
    
    # 2018 and earlier files have column names capitalized
    grad.columns = grad.columns.str.lower()
    
    grad = grad[[
        "report_school_year",
        "aggregation_code",  # ID for school district
        "aggregation_index",
        "subgroup_code",
        "membership_code",
        "grad_pct",
        "reg_adv_pct" 
    ]]

    grad_filt = grad[
        (grad["aggregation_index"] == 3) &  # District
        (grad["subgroup_code"] == 1) &  # All Students
        (grad["membership_code"] == 6)  # Up to 6 years to graduate
    ].reset_index(drop=True)

    for col in ["grad_pct", "reg_adv_pct"]:
        fix_percent(grad_filt, col)

    grad_filt.drop(["aggregation_index", "subgroup_code", "membership_code"], 
                   axis=1,
                   inplace=True)

    grad_filt.columns = ["Year", "aggregation_code", "Graduation Rate", "Advanced Regents Diploma Rate"]

    return grad_filt
    

In [19]:
def clean_older_data(file: str):

    grad = pd.read_excel(file, sheet_name="outcomes districts", engine="openpyxl")

    grad = grad[[
        "REPORT_SCHOOL_YEAR",
        "AggrCode",
        "SUBGROUP_CODE",
        "Graduate%",
        "%RegentsAdvancedDesig%",
        "MEMBERSHIP_DESC"
    ]]

    grad = grad.rename(columns={
        "REPORT_SCHOOL_YEAR": "report_school_year",
        "AggrCode": "aggregation_code",
        "SUBGROUP_CODE": "subgroup_code",
        "Graduate%": "grad_pct",
        "%RegentsAdvancedDesig%": "reg_adv_pct"
    })

    # Looking specificlly for 6 year outcome and anything else set to -1
    pattern = r'6 Year Outcome'
    grad['membership_code'] = np.where(grad['MEMBERSHIP_DESC'].str.contains(pattern, regex=True), 6, -1)

    grad_filt = grad[
        (grad["subgroup_code"] == 1) &  # All Students
        (grad["membership_code"] == 6)  # Up to 6 years to graduate
    ].reset_index(drop=True)

    for col in ["grad_pct", "reg_adv_pct"]:
        fix_percent(grad_filt, col)

    grad_filt.drop(["subgroup_code", "membership_code", "MEMBERSHIP_DESC"], 
                   axis=1,
                   inplace=True)

    grad_filt.columns = ["Year", "aggregation_code", "Graduation Rate", "Advanced Regents Diploma Rate"]

    return grad_filt

In [20]:
# List of files to process
files = [
    # Manually changed this first file from .xls to .xlsx
    "raw/201112-GradRateRelease-GradsDiplomasOutcomes.xlsx",
    "raw/201213-GradRateReleaseOutcomes.xlsx",
    "raw/Suppressed 201314GraduationRateAndEnrollmentOutcomes.xlsx",
    "raw/GRAD_RATE_AND_OUTCOMES_2015.csv",
    "raw/GRAD_RATE_AND_OUTCOMES_2016.csv",
    "raw/GRAD_RATE_AND_OUTCOMES_2017.csv",
    "raw/GRAD_RATE_AND_OUTCOMES_2018.csv",
    "raw/GRAD_RATE_AND_OUTCOMES_2019.csv",
    "raw/GRAD_RATE_AND_OUTCOMES_2020.csv",
    "raw/GRAD_RATE_AND_OUTCOMES_2021.csv",
    "raw/GRAD_RATE_AND_OUTCOMES_2022.csv"
]

# Collect cleaned DataFrames
frames = []

for file in files:
    if re.search(r'GRAD_RATE_AND_OUTCOMES', file):
        grad_filt = clean_newer_data(file)
    else:
        grad_filt = clean_older_data(file)

    frames.append(grad_filt)

# Combine all years
grad_all_years = pd.concat(frames, ignore_index=True)

/tmp/ipykernel_1480502/3702334326.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grad_filt[col] = grad_filt[col].replace(["-", "#"], value=np.nan)
/tmp/ipykernel_1480502/3702334326.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grad_filt[col] = grad_filt[col].replace(["-", "#"], value=np.nan)
/tmp/ipykernel_1480502/1393585950.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  grad = pd.read_csv(file)
/tmp/ipykernel_1480502/1393585950.py:3: DtypeWarning: Columns (5,

In [21]:
# Save to file
grad_all_years.to_csv("clean/graduation.csv", index=False)

### III. NCED-NYSED Crosswalk

Source: https://nces.ed.gov/ccd/districtsearch/
- (Search for only 'New York')

In [22]:
# Manually had to change this file from .xls to .xlsx
crosswalk = pd.read_excel("raw/ncesdata_D1351086.xlsx",
                          engine="openpyxl",
                          skiprows=11)

crosswalk = crosswalk[["NCES District ID", "State District ID"]]
crosswalk["aggregation_code"] = crosswalk["State District ID"].str.extract(r'NY-(\d+)')
crosswalk["aggregation_code"] = pd.to_numeric(crosswalk["aggregation_code"])
crosswalk.drop(columns=["State District ID"], inplace=True)
crosswalk = crosswalk.rename(columns={
        "NCES District ID": "nces_id"})

crosswalk.to_csv("clean/crosswalk.csv", index=False)

### IV. School District Shapefiles

Source: https://data.gis.ny.gov/datasets/sharegisny::school-districts/explore

Backup Source: https://hub.arcgis.com/datasets/nces::school-district-boundaries-current/about

In [23]:
gdf = gpd.read_file("raw/NYS_Schools_3089706237687211859.geojson")

In [24]:
print(gdf.columns)
print(gdf.head())
# SED_CODE_1 is what we want to merge on because it matches aggregation_code

Index(['OBJECTID', 'SCHOOL_ID', 'SCHOOLDIST', 'POLYTYPE', 'PRIMARYPOL',
       'USERNAME', 'UPDATE_', 'EDITED', 'EACODE', 'District2', 'Inst_ID',
       'SchDist', 'SEDdir_BOC', 'SEDdir_B_1', 'SED_CODE_1', 'POPULAR_NA',
       'INSTPECD', 'INSTSUBYPE', 'INSSUBDE', 'MUNICODE', 'SDLCODE',
       'ORPTSCOD_1', 'geometry'],
      dtype='object')
   OBJECTID  SCHOOL_ID   SCHOOLDIST      POLYTYPE PRIMARYPOL USERNAME  \
0         1        288  SO MOUNTAIN      DTM ONLY          Y  unknown   
1         2          2   LONG BEACH      DTM ONLY          Y  unknown   
2         3          3  ISLAND PARK      DTM ONLY          Y  unknown   
3         4          4  BROOKLYN 21  NON-DTM ONLY          Y  unknown   
4         5          5  BROOKLYN 22  NON-DTM ONLY          Y  unknown   

                         UPDATE_ EDITED  EACODE    District2  ...  \
0  Mon, 01 Jan 1900 00:00:00 GMT   NONE  032201  SO MOUNTAIN  ...   
1  Mon, 01 Jan 1900 00:00:00 GMT   NONE  281000   LONG BEACH  ...   
2  Mon, 01

In [25]:
gdf["SED_CODE_1"] = pd.to_numeric(gdf["SED_CODE_1"], errors="coerce")

# Interactive Data Filtering

In [26]:
district.head()

,Year,ansi_fips,nces_id,School District,County Name,State,State Name,Agency Type,School District Level Code (SCHLEV),Locale,...,Percent Male Students,Percent Female Students,Percent Limited English Proficient (LEP) / English Language Learners (ELL),Percent White Students,Percent Black or African American Students,Percent American Indian/Alaska Native Students,Percent Asian or Asian/Pacific Islander Students,Percent Hispanic Students,Percent Nat. Hawaiian or Other Pacific Isl. Students,Percent Two or More Races Students
0,2021-22,36,3602300,ABBOTT UNION FREE SCHOOL DISTRICT,Westchester County,NY,New York,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-22,36,3601001,ACADEMIC LEADERSHIP CHARTER SCHOOL,Bronx County,NY,New York,7-Independent Charter District,01-Elementary school system only,11-City: Large,...,0.511745,0.488255,NaN,0.020134,0.429530,0.008389,0.006711,0.523490,NaN,0.011745
2,2021-22,36,3600997,ACADEMY CHARTER SCHOOL,Nassau County,NY,New York,7-Independent Charter District,03-Elementary/secondary school system,21-Suburb: Large,...,0.460583,0.539417,NaN,0.005940,0.433585,0.013499,0.007019,0.531317,0.001620,0.007019
3,2021-22,36,3601169,ACADEMY CHARTER SCHOOL-UNIONDALE,Nassau County,NY,NEW YORK,7-Independent Charter District,03-Elementary/secondary school system,21-Suburb: Large,...,0.508056,0.491944,NaN,0.002148,0.614393,0.010741,0.006445,0.353383,0.001074,0.011815
4,2021-22,36,3601190,ACADEMY OF HEALTH SCIENCES CHARTER SCHOOL,Monroe County,NY,NEW YORK,7-Independent Charter District,01-Elementary school system only,12-City: Mid-size,...,0.521898,0.478102,NaN,0.054745,0.777372,0.003650,NaN,0.164234,NaN,NaN


In [27]:
grad_all_years.head()

,Year,aggregation_code,Graduation Rate,Advanced Regents Diploma Rate
0,2011-12,10100010000,0.569,0.139
1,2011-12,10201040000,0.876,0.361
2,2011-12,10306060000,0.942,0.609
3,2011-12,10402060000,0.813,0.295
4,2011-12,10500010000,0.720,0.214


In [28]:
crosswalk.head()

,nces_id,aggregation_code
0,3601001,320700860957
1,3600997,280201860934
2,3680973,580109861195
3,3601169,280202861142
4,3601190,261600861153


In [29]:
district.columns

Index(['Year', 'ansi_fips', 'nces_id', 'School District', 'County Name',
       'State', 'State Name', 'Agency Type',
       'School District Level Code (SCHLEV)', 'Locale', 'Latitude',
       'Longitude', 'State Agency ID', 'census_id', 'Highest Grade Offered',
       'Total Students All Grades (Excludes AE)',
       'Full-Time Equivalent (FTE) Teachers', 'Pupil/Teacher Ratio',
       'Paraprofessionals/Instructional Aides',
       'LEA Administrative Support Staff', 'School Administrators',
       'Total Revenue (TOTALREV)', 'Total Revenue - Local Sources (TLOCREV)',
       'Total Revenue - State Sources (TSTREV)',
       'Total Revenue - Federal Sources (TFEDREV)',
       'Total Current Expenditures - Instruction (TCURINST)',
       'Total Current Expenditures - Support Services (TCURSSVC)',
       'Total Current Expenditures - Salary (Z32)',
       'Total Expenditures (TOTALEXP)',
       'Total Expenditures - Capital Outlay (TCAPOUT)',
       'Total Current Expenditures - Instructi

In [30]:
d = district[[
        "Year",
        "School District",
        "County Name",
        "Pupil/Teacher Ratio",
        "Total Revenue (TOTALREV)",
        "Total Revenue - Local Sources (TLOCREV)",
        "Total Revenue - State Sources (TSTREV)",
        "Total Revenue - Federal Sources (TFEDREV)",
        "Total Expenditures (TOTALEXP)",
        "Total Expenditures (TOTALEXP) per Pupil (V33)",
        "Total Current Expenditures - Instruction (TCURINST) per Pupil (V33)",
        "Total Current Expenditures - Support Services (TCURSSVC) per Pupil (V33)",
        "nces_id",
        "Highest Grade Offered",
        "Locale"
    ]].copy()

d.rename(columns={
    "Year": "year",
    "School District": "district",
    "County Name": "county",
    "Pupil/Teacher Ratio": "pupil_teacher",
    "Total Revenue (TOTALREV)": "revenue",
    "Total Revenue - Local Sources (TLOCREV)": "revenue_local",
    "Total Revenue - State Sources (TSTREV)": "revenue_state",
    "Total Revenue - Federal Sources (TFEDREV)": "revenue_federal",
    "Total Expenditures (TOTALEXP)": "expenditure",
    "Total Expenditures (TOTALEXP) per Pupil (V33)": "expenditure_pupil",
    "Total Current Expenditures - Instruction (TCURINST) per Pupil (V33)": "expenditure_instruction_pupil",
    "Total Current Expenditures - Support Services (TCURSSVC) per Pupil (V33)": "expenditure_support_pupil",
    "Highest Grade Offered": "highest_grade"
}, inplace=True)

d["enrollment"] = d["expenditure"]/d["expenditure_pupil"]
d["revenue_pupil"] = d["revenue"]/d["enrollment"]
d["revenue_local_pupil"] = d["revenue_local"]/d["enrollment"]
d["revenue_state_pupil"] = d["revenue_state"]/d["enrollment"]
d["revenue_federal_pupil"] = d["revenue_federal"]/d["enrollment"]

In [31]:
def make_locale_group(df: pd.DataFrame):
    return df["Locale"].str.extract(r"(City|Suburb|Town|Rural)", expand=False)

d["locale_group"] = make_locale_group(d)

d.drop('Locale', axis=1, inplace=True)

In [32]:
new_order = ["year",
             "district",
             "county",
             "locale_group",
             "highest_grade",
             "enrollment",
             "pupil_teacher",
             "revenue",
             "revenue_local",
             "revenue_state",
             "revenue_federal",
             "revenue_pupil",
             "revenue_local_pupil",
             "revenue_state_pupil",
             "revenue_federal_pupil",
             "expenditure",
             "expenditure_pupil",
             "expenditure_instruction_pupil",
             "expenditure_support_pupil",
             "nces_id"]
d = d[new_order]

In [33]:
g = grad_all_years[[
        "Year",
        "aggregation_code",
        "Graduation Rate",
        "Advanced Regents Diploma Rate"
    ]].copy()

g.rename(columns={
    "Year": "year",
    "Graduation Rate": "grad_rate",
    "Advanced Regents Diploma Rate": "ard_rate",
    "aggregation_code": "SED_CODE_1"
}, inplace=True)

In [34]:
crosswalk.rename(columns={
    "aggregation_code": "SED_CODE_1"
}, inplace=True)

In [35]:
# def twelfth_grade_only(df: pd.DataFrame):
#     return df[df["Highest Grade Offered"] == "12th Grade"]

# def drop_nulls(df: pd.DataFrame, cols: list):
#     return df.dropna(subset=cols)

# def remove_zeros(df: pd.DataFrame, cols: list):
#     for col in cols:
#         df = df[df[col] != 0]
#     return df

# d = twelfth_grade_only(d)

# d.drop('Highest Grade Offered', axis=1, inplace=True)

# g = drop_nulls(g, ["grad_rate"])

# Data Merging

In [36]:
# I want to keep all graduation rate data so I'm going to left join into that
merge1 = pd.merge(d, crosswalk, how="left", on="nces_id")
merge2 = pd.merge(merge1, g, how="left", on=["SED_CODE_1", "year"])

merge2.to_csv("interactive_data.csv", index=False)

In [37]:
# merge3 = pd.merge(merge2,
#                   gdf[["geometry","SED_CODE_1"]],
#                   how="left",
#                   left_on="aggregation_code",
#                   right_on="SED_CODE_1")

In [38]:
# # Convert to GeoDataFrame
# merge3_geo = gpd.GeoDataFrame(merge3, geometry="geometry", crs=gdf.crs)

# # Export to GeoJSON
# merge3_geo.to_file("interactive_data.geojson", driver="GeoJSON")